## import packages

In [122]:
## importing packages

import requests
import pandas as pd
from sqlalchemy import create_engine

## extract data

In [123]:
def extract(country="United States")-> dict: # here I have set default parameter value to "United States"
    # API Call
    api_url = f"http://universities.hipolabs.com/search?country={country}"
    data = requests.get(api_url).json()
    print(f"Total number of universities in {country}: {len(data)}")
    return data

## transform data

In [124]:
def transform(data, city="California")-> pd.DataFrame: # default to "California"
    df = pd.DataFrame(data)

    # Filtering entire dataframe with name having city value
    df = df[df["name"].str.contains(f"{city}")]
    
    print(f"Total number of universities in {city}: {len(df)}")
    
    # Here I use list comprehension that iterates through each value, and for each value it converts each element in the list to a string. 
    # Finally returns a single string where elements are now seperated by the comma.
    df['domains'] = [','.join(map(str, l)) for l in df['domains']]
    df['web_pages'] = [','.join(map(str, l)) for l in df['web_pages']]

    df = df.reset_index(drop=True)
    
    # Exported Coloumns
    return df[['domains','country','web_pages','name']]


## load data

In [125]:
def load(df)-> None:
    # Loads data into a sqlite database
    disk_engine = create_engine('sqlite:///universities.db')
    df.to_sql('uni_table', disk_engine, if_exists='replace')

## calling functions

In [126]:
ext_data = extract() # optional - Enter Country as arg to search
dataframe = transform(ext_data) # optional - Enter City as second arg to search
load(dataframe)
dataframe

Total number of universities in United States: 4535
Total number of universities in California: 84


,domains,country,web_pages,name
0,csuci.edu,United States,https://www.csuci.edu/,California State University Channel Islands
1,calbaptist.edu,United States,http://www.calbaptist.edu/,California Baptist University
2,calsouthern.edu,United States,http://www.calsouthern.edu/,California Southern University
3,cpp.edu,United States,http://www.cpp.edu/,"California Polytechnic State University, Pomona"
4,calcoast.edu,United States,http://www.calcoast.edu/,California Coast University
...,...,...,...,...
79,ucsb.edu,United States,http://www.ucsb.edu/,"University of California, Santa Barbara"
80,ucsc.edu,United States,http://www.ucsc.edu/,"University of California, Santa Cruz"
81,usc.edu,United States,http://www.usc.edu/,University of Southern California
82,carrington.edu,United States,http://www.carrington.edu,Carrington College California-​Sacramento
